# GPT2-Small Resid Pre

In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('jbloom/mats_sae_training_gpt2_small_resid_pre/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28', type='model')
# artifact_dir = artifact.download()

In [ ]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
    fold_ln=False,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

## Get Sparsity

In [ ]:
from tqdm.auto import tqdm
def estimate_feature_sparsity_using_n_tokens_per_prompt(
    sparse_autoencoder, activation_store, n_batches,
    n_tokens_per_prompt=4):
    
    total_activations = torch.zeros(sparse_autoencoder.cfg.d_sae).to(sparse_autoencoder.cfg.device)
    
    pbar = tqdm(range(n_batches))
    for _ in pbar:
        batch_tokens = activation_store.get_batch_tokens()
        _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
        original_act = cache[sparse_autoencoder.cfg.hook_point]
        _, feature_acts, _, _, _ = sparse_autoencoder(
            original_act
        )
        # for each batch item, pick 4 random tokens and keep only those
        # batch_size x n_tokens x d_sae
        random_tok_indices = torch.randint(0, feature_acts.shape[1], (feature_acts.shape[0], n_tokens_per_prompt))
        feature_acts = feature_acts[torch.arange(feature_acts.shape[0]).unsqueeze(-1), random_tok_indices]
        total_activations += feature_acts.flatten(0,1).sum(0)
    
    total_tokens = (n_batches * feature_acts.shape[0] * n_tokens_per_prompt)
    print("Total tokens:", total_tokens)
    
    return total_activations / total_tokens


# feature_sparsity = estimate_feature_sparsity_using_n_tokens_per_prompt(sparse_autoencoder, activation_store, 5000).detach().cpu()
# log_feature_sparsity = torch.log10(feature_sparsity + 1e-10)
# torch.save(log_feature_sparsity, "./artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/log_feature_sparsity_5000_4.pt")

# log_feature_sparsity = torch.load("./artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/log_feature_sparsity_5000_4.pt")
log_feature_sparsity = torch.load("../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/log_feature_sparsity_5000_4.pt")

px.histogram(log_feature_sparsity[log_feature_sparsity>-9], nbins=100, log_x=False, title="Feature sparsity (log10) (5000 batches, 4 tokens per prompt)").show()
px.histogram(log_feature_sparsity, nbins=100, log_x=False, title="Feature sparsity (log10) (5000 batches, 4 tokens per prompt)").show()

In [ ]:
(log_feature_sparsity == -10).sum() / log_feature_sparsity.shape[0]

In [ ]:
W_enc_normalized = sparse_autoencoder.W_enc / sparse_autoencoder.W_enc.norm(dim=-2, keepdim=True)

In [ ]:
import hdbscan

def get_sae_df(sparse_autoencoder, min_cluster_size=30):
    
    W_enc_normalized = sparse_autoencoder.W_enc / sparse_autoencoder.W_enc.norm(dim=-2, keepdim=True)
    d_e_projection = (sparse_autoencoder.W_dec @ sparse_autoencoder.W_enc).diag().detach().cpu()
    # px.histogram(d_e_projection, nbins=100, log_x=False, title="d_e projection").show()


    d_e_projection_normalized = (sparse_autoencoder.W_dec @ W_enc_normalized).diag().detach().cpu()
    # px.histogram(d_e_projection_normalized, nbins=100, log_x=False, title="d_e projection").show()
    
    temp_df = pd.DataFrame({
    "log_feature_sparsity": log_feature_sparsity,
    "d_e_projection": d_e_projection,
    "d_e_projection_normalized": d_e_projection_normalized,
    "b_enc": sparse_autoencoder.b_enc.detach().cpu(),
    "feature": [f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)],
    "index": torch.arange(sparse_autoencoder.cfg.d_sae)
    })

    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    clusterer.fit(temp_df[["log_feature_sparsity", "d_e_projection", "b_enc"]].values)
    temp_df["cluster"] = clusterer.labels_
    temp_df["cluster_categorical"] = temp_df["cluster"].astype(str)
    temp_df["outlier"] = (temp_df.log_feature_sparsity < -9) | (temp_df.cluster == -1)
    
    return temp_df

sae_df = get_sae_df(sparse_autoencoder)
display(sae_df.head())

px.bar(sae_df.cluster.value_counts().reset_index(), y = "count", color = "cluster", title = "Cluster sizes").show()


def get_anthropic_3d_scatter(temp_df):

    fig = px.scatter_3d(
        # temp_df,#[~temp_df.outlier],
        temp_df[~temp_df.outlier],
        x="log_feature_sparsity",
        y="d_e_projection",
        z="b_enc",
        color="cluster",
        # color="index",
        hover_name="feature",
    )
    fig.update_layout(
        scene = dict(
            xaxis_title='log10 feature sparsity',
            yaxis_title='D/E Projection',
            zaxis_title='Encoder Bias',
        )
    )

    fig.update_traces(marker_size = 2)
    fig.update_layout(
        title="Feature Sparsity vs D/E projection vs Encoder Bias (Excluding Outliers)",
        width=1400,
        height=1300,
    ) 

    # increase font size everywhere
    fig.update_layout(
        font=dict(
            size=18,
        )
    )

    # increase marker size in legend (only)
    fig.update_layout(legend= {'itemsizing': 'constant'})


    return fig

fig = get_anthropic_3d_scatter(sae_df)
fig.show()

In [ ]:
# px.scatter_matrix(
#     temp_df[~temp_df.outlier],
#     dimensions = ["log_feature_sparsity", "d_e_projection", "b_enc"],
#     color="cluster",
#     hover_name="feature",
#     opacity=0.3,
#     height=1400,
#     width =1400,
# )

# px.parallel_coordinates(
#     temp_df[~temp_df.outlier],
#     color="cluster",
#     dimensions = ["log_feature_sparsity", "d_e_projection", "b_enc","cluster", "index"],
#     # color_continuous_scale=px.colors.diverging.Tealrose,
#     height=1400,
# )

# for cluster in temp_df[~temp_df.outlier].cluster.unique():
#     print(f"Cluster {cluster}")
#     display(temp_df[temp_df.cluster == cluster].sample(3))


# px.histogram(
#     temp_df[~temp_df.outlier], 
#     x="b_enc", color="cluster", title="Cluster sizes", barmode="overlay",
#     histnorm="percent",)

# Weight Figs

In [ ]:
alive_features = (log_feature_sparsity > -8).nonzero()
print(len(alive_features) / log_feature_sparsity.shape[0])

dead_features = (log_feature_sparsity <= -8).nonzero()
print(len(dead_features) / log_feature_sparsity.shape[0])

dense_mode_features = ((log_feature_sparsity > -3.5) & (log_feature_sparsity < -1) ).nonzero()
print(len(dense_mode_features) / log_feature_sparsity.shape[0])

sparse_mode_features = ((log_feature_sparsity > -7) & (log_feature_sparsity < -4) ).nonzero()
print(len(sparse_mode_features) / log_feature_sparsity.shape[0])


W_dec_alive_features = sparse_autoencoder.W_dec[alive_features].squeeze(1)
W_dec_dead_features = sparse_autoencoder.W_dec[dead_features].squeeze(1)
W_dec_dense_mode_features = sparse_autoencoder.W_dec[dense_mode_features].squeeze(1)
W_dec_sparse_mode_features = sparse_autoencoder.W_dec[sparse_mode_features].squeeze(1)


In [ ]:
print(layer)

In [ ]:


def get_feature_weight_proj(W_dec_features, feature_indices, layer=10):

    layer = sparse_autoencoder.cfg.hook_point_layer
    W_QK_10 = torch.stack(
        [model.W_Q[layer,i] @ model.W_K[layer,i].T for i in range(model.cfg.n_heads)],
        dim=0
    )


    # query_proj_dec_weights = sparse_autoencoder.W_dec @ W_QK_10
    query_proj_dec_weights = W_dec_features @ W_QK_10
    # query_proj_dec_weights = W_dec_features @ model.W_Q[10,:]
    query_proj_dec_weights_df = pd.DataFrame(
        query_proj_dec_weights.norm(dim=-1).detach().cpu().numpy().T,
        columns = [f"L{layer}H" + str(i) for i in range(12)],
        index = [f"feature_{i}" for i in feature_indices])

    # key_proj_dec_weights = sparse_autoencoder.W_dec @ W_QK_10.transpose(2,1)
    key_proj_dec_weights = W_dec_features @ W_QK_10.transpose(2,1)
    # key_proj_dec_weights = W_dec_features @ model.W_K[10,:]
    key_proj_dec_weights_df = pd.DataFrame(
        key_proj_dec_weights.norm(dim=-1).detach().cpu().numpy().T,
        columns = [f"L{layer}H" + str(i) for i in range(12)],
        index = [f"feature_{i}" for i in feature_indices])
    return query_proj_dec_weights_df, key_proj_dec_weights_df




def get_head_proj_norm_histograms(proj_dec_weights_df, title = ""):

    fig = px.histogram(
        proj_dec_weights_df,
        x=proj_dec_weights_df.columns,
        facet_col='variable',
        facet_col_wrap=3,
        # barmode='overlay',
        title = title,
    )
    
    return fig



# print("All Alive Features")
# query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_alive_features, alive_features)
print("Dense Mode Features")
query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_dense_mode_features, dense_mode_features)
get_head_proj_norm_histograms(query_proj_dec_weights_df, title = "Query Projection Norms").show()
get_head_proj_norm_histograms(key_proj_dec_weights_df, title = "Key Projection Norms").show()

print("Sparse Mode Features")
query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_sparse_mode_features, sparse_mode_features)
get_head_proj_norm_histograms(query_proj_dec_weights_df, title = "Query Projection Norms").show()
get_head_proj_norm_histograms(key_proj_dec_weights_df, title = "Key Projection Norms").show()


cluster_feature_ids = {}
cluster_wise_features = {}

for cluster in [16]:#sae_df[~sae_df.outlier].cluster.unique():

    cluster_x_features = sae_df[(sae_df.cluster == cluster)].index.values
    cluster_feature_ids[cluster] = cluster_x_features
    
    print(f"Cluster {cluster}", len(cluster_x_features) / log_feature_sparsity.shape[0] , len(cluster_x_features))
    
    W_dec_cluster_x_features = sparse_autoencoder.W_dec[cluster_x_features].squeeze(1)
    cluster_wise_features[cluster] = W_dec_cluster_x_features
    
    query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_cluster_x_features, cluster_x_features)
    get_head_proj_norm_histograms(query_proj_dec_weights_df, title = "Query Projection Norms").show()
    get_head_proj_norm_histograms(key_proj_dec_weights_df, title = "Key Projection Norms").show()

In [ ]:
for cluster in sae_df[~sae_df.outlier].cluster.unique():
    print(f"Cluster {cluster}")
    display(sae_df[sae_df.cluster == cluster].sample(3))

In [ ]:
for cluster in sae_df[~sae_df.outlier].cluster.unique():
    print(f"Cluster {cluster}")
    query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(cluster_wise_features[cluster], cluster_feature_ids[cluster])
    get_head_proj_norm_histograms(query_proj_dec_weights_df, title = "Query Projection Norms").show()
    get_head_proj_norm_histograms(key_proj_dec_weights_df, title = "Key Projection Norms").show()


In [ ]:
layer = 10
print(model.W_Q.shape)
W_QK_10 = torch.stack(
    [model.W_Q[layer,i] @ model.W_K[layer,i].T for i in range(model.cfg.n_heads)],
    dim=0
)

# calculate the singular values of the query projection matrix
U, S, V = torch.svd(W_QK_10)
print(S.shape)
print(U.shape)
print(V.shape)

In [ ]:
px.line((cluster_wise_features[16] @ W_QK_10).norm(dim=-1).T.detach().cpu()).show()
px.line((cluster_wise_features[16] @ W_QK_10.transpose(2,1)).norm(dim=-1).T.detach().cpu()).show()

In [ ]:
(cluster_wise_features[16] @ W_QK_10).T.shape

In [ ]:
cluster_wise_features[16].shape

In [ ]:
(V @ cluster_wise_features[16].T).shape

In [ ]:
from IPython.core.display import display, HTML

path_to_html = "../week_8_jan/gpt2_small_features"
def render_feature_dashboard(feature_id):
    
    path = f"{path_to_html}/data_{feature_id}.html"
    
    
    print(f"Feature {feature_id}")
    
    if os.path.exists(path):
        # with open(path, "r") as f:
        #     html = f.read()
        #     display(HTML(html))
        webbrowser.open_new_tab("file://" + os.path.abspath(path))
    else:
        print("No HTML file found")
    
    return

for feature in sae_df[sae_df.cluster == 16].sample(5).index.values:
    render_feature_dashboard(feature)

In [ ]:
sae_df.iloc[22827]

In [ ]:
query_proj_dec_weights_df_16.index = temp_df_16.index.astype(str).str.replace("feature_", "")
query_proj_dec_weights_df_16

In [ ]:
temp_df_16.head()

In [ ]:
fig = px.scatter_matrix(
    query_proj_dec_weights_df_16,
    dimensions=query_proj_dec_weights_df_16.columns,
    color=log_feature_sparsity[temp_df_16.index],
    # color=range(sparse_autoencoder.cfg.d_sae),
    color_continuous_scale='RdBu',
    opacity=0.3,
    hover_data=[query_proj_dec_weights_df_16.index],
)
fig.update_layout(width = 1800, height = 1800)
fig.show()

In [ ]:
fig = px.scatter_matrix(
    key_proj_dec_weights_df_16,
    dimensions=key_proj_dec_weights_df_16.columns,
    color=log_feature_sparsity[temp_df_16.index],
    # color=range(sparse_autoencoder.cfg.d_sae),
    color_continuous_scale='RdBu',
    opacity=0.3,
    hover_data=[key_proj_dec_weights_df_16.index],
)
fig.update_layout(width = 1800, height = 1800)
fig.show()

In [ ]:
fig = px.scatter(
    y = key_proj_dec_weights_df_16["L10H7"],
    x=log_feature_sparsity[temp_df_16.index],
    # color=range(sparse_autoencoder.cfg.d_sae),
    color_continuous_scale='RdBu',
    opacity=0.3,
    hover_data=[key_proj_dec_weights_df_16.index],
)
fig.show()

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.01,
    n_components=2,
    metric="cosine",
    random_state=42,
)


ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec.detach().cpu())
temp_df["ummap_x"] = ummap_result[:,0]
temp_df["ummap_y"] = ummap_result[:,1]

In [ ]:
fig = px.scatter(
        temp_df,#[~temp_df.outlier],
        x="ummap_x",
        y="ummap_y",
        color="cluster_categorical",
        # color_continuous_midpoint=0,
        # color_continuous_scale="RdBu",
        hover_name="feature",
        opacity=0.5,
        template="plotly",
    )

# make points larger
# fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
    
fig.show()

In [ ]:
temp_df.iloc[[
    14627,44798,6259, 
]]

In [ ]:
query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_sparse_mode_features, sparse_mode_features)

In [ ]:
 temp_df[~temp_df.outlier].cluster.unique()

In [ ]:

# x = 13
# cluster_13_features = temp_df[(temp_df.cluster == 13)].index.values
# print(len(cluster_13_features) / log_feature_sparsity.shape[0])
# print(len(cluster_13_features))

# cluster_10_features = temp_df[(temp_df.cluster == 10)].index.values
# print(len(cluster_10_features) / log_feature_sparsity.shape[0])
# print(len(cluster_10_features))

# W_dec_cluster_10_features = sparse_autoencoder.W_dec[cluster_13_features].squeeze(1)
W_dec_cluster_13_features = sparse_autoencoder.W_dec[cluster_10_features].squeeze(1)

lines = []
for cluster in temp_df[~temp_df.outlier].cluster.unique():
    feature_idxs =  temp_df[(temp_df.cluster == cluster)].index.values
    # features =  sparse_autoencoder.W_dec[cluster_13_features]
    print(sparse_autoencoder.W_dec[feature_idxs].mean(0).detach().cpu().shape)
    lines.append(
        sparse_autoencoder.W_dec[feature_idxs].mean(0).detach().cpu()
    )


lines = torch.stack(lines).T
px.line(lines)

In [ ]:
for cluster in temp_df[~temp_df.outlier].cluster.unique():
    # get the first 10 features
    cluster_x_features = temp_df[(temp_df.cluster == cluster)].index.values
    px.line(sparse_autoencoder.W_dec[cluster_x_features].mean(0).detach().cpu(), title=f"Cluster {cluster} mean W_dec").show()

In [ ]:
px.imshow(query_proj_dec_weights_df.corr(), color_continuous_scale='RdBu', title = "Query projection weights", color_continuous_midpoint=0).show()
px.imshow(key_proj_dec_weights_df.corr(), color_continuous_scale='RdBu', title = "Key projection weights", color_continuous_midpoint=0).show()
# px.imshow(value_proj_dec_weights_df.corr(), color_continuous_scale='RdBu', title = "Value projection weights").show()

In [ ]:
px.imshow(query_proj_dec_weights_df_16.corr(), color_continuous_scale='RdBu', title = "Query projection weights", color_continuous_midpoint=0).show()
px.imshow(key_proj_dec_weights_df_16.corr(), color_continuous_scale='RdBu', title = "Key projection weights", color_continuous_midpoint=0).show()

In [ ]:
tmp = W_QK_10 @ sparse_autoencoder.b_dec
px.bar(tmp.norm(dim=-1).detach().cpu().numpy().T, title="Query projection onto decoder bias").show()

In [ ]:
tmp = W_QK_10.transpose(2,1) @ sparse_autoencoder.b_dec
px.bar(tmp.norm(dim=-1).detach().cpu().numpy().T, title="Query projection onto decoder bias").show()
px.scatter(
    y = (W_QK_10.transpose(2,1) @ sparse_autoencoder.b_dec).norm(dim=-1).detach().cpu().numpy().T,
    x = (W_QK_10 @ sparse_autoencoder.b_dec).norm(dim=-1).detach().cpu().numpy().T,
    hover_name = [f"L10H{i}" for i in range(12)],
)

In [ ]:
# query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_alive_features, alive_features)
query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_sparse_mode_features, sparse_mode_features)
query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(W_dec_sparse_mode_features, sparse_mode_features)

fig = px.scatter_matrix(
    query_proj_dec_weights_df,
    dimensions=query_proj_dec_weights_df.columns,
    color=log_feature_sparsity,
    # color=range(sparse_autoencoder.cfg.d_sae),
    color_continuous_scale='RdBu',
    opacity=0.3,
    hover_data=[query_proj_dec_weights_df.index],
)
fig.update_layout(width = 1800, height = 1800)
fig.show()

# fig = px.scatter_matrix(
#     key_proj_dec_weights_df,
#     dimensions=key_proj_dec_weights_df.columns,
#     color=range(sparse_autoencoder.cfg.d_sae),
#     # color=log_feature_sparsity,
#     opacity=0.3,
#     hover_data=[key_proj_dec_weights_df.index],
# )
# fig.update_layout(width = 1800, height = 1800)
# fig.show()


In [ ]:

fig = px.scatter(
    x = query_proj_dec_weights_df["L10H5"],
    y = log_feature_sparsity,
    hover_name=query_proj_dec_weights_df.index,
    opacity=0.5,
    marginal_x="histogram",
    marginal_y="histogram",
    labels={
        "x": "Feature Norm after Query Projection",
        "y": "Feature Norm after Key Projection",
        "color": "Sparsity",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:
px.line(
    query_proj_dec_weights_df
)

# Investigate first 8192 features

In [ ]:
px.line(sparse_autoencoder.W_dec.norm(dim=-1).detach().cpu())

In [ ]:
px.line(sparse_autoencoder.W_enc.norm(dim=0).detach().cpu())

In [ ]:
sparse_autoencoder.W_enc.norm(dim=0, keepdim=True).shape

In [ ]:
normalized_W_enc = sparse_autoencoder.W_enc / sparse_autoencoder.W_enc.norm(dim=0, keepdim=True)

sim = normalized_W_enc.T @ normalized_W_enc
print(sim.shape)
px.imshow(sim[:1000,:1000].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Encoder weight similarity (first 1000)").show()
print(sim[:1000,:1000].detach().cpu().numpy().flatten().mean())
px.imshow(sim[-1000:,-1000:].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Encoder weight similarity (last 1000)").show()
print(sim[-1000:,-1000:].detach().cpu().numpy().flatten().mean())
px.imshow(sim[:1000,-1000:].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Encoder weight similarity (first 1000 to last 1000)").show()
print(sim[:1000,-1000:].detach().cpu().numpy().flatten().mean())

In [ ]:
sim = sparse_autoencoder.W_dec @ sparse_autoencoder.W_dec.T
print(sim.shape)
px.imshow(sim[:1000,:1000].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Decoder weight similarity (first 1000)").show()
print(sim[:1000,:1000].detach().cpu().numpy().flatten().mean())
px.imshow(sim[-1000:,-1000:].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Decoder weight similarity (last 1000)").show()
print(sim[-1000:,-1000:].detach().cpu().numpy().flatten().mean())
px.imshow(sim[:1000,-1000:].detach().cpu().numpy(), color_continuous_scale='RdBu', title = "Decoder weight similarity (first 1000 to last 1000)").show()
print(sim[:1000,-1000:].detach().cpu().numpy().flatten().mean())

In [ ]:
px.line(sparse_autoencoder.b_dec.detach().cpu())

In [ ]:
unit_norm_b_dec = sparse_autoencoder.b_dec / sparse_autoencoder.b_dec.norm(dim=-1, keepdim=True)
px.line(unit_norm_b_dec.detach().cpu())

In [ ]:
px.line((sparse_autoencoder.W_dec[:500] @ unit_norm_b_dec).detach().cpu())

In [ ]:
px.scatter(
    x = range(sparse_autoencoder.cfg.d_sae),
    y = (sparse_autoencoder.W_dec @ unit_norm_b_dec).detach().cpu(),
    marginal_y="histogram",
)

In [ ]:
centred_W_dec = sparse_autoencoder.W_dec  - sparse_autoencoder.W_dec.mean(dim=0, keepdim=True)

In [ ]:
cosine_similarity_with_bias = sparse_autoencoder.W_dec @ unit_norm_b_dec
most_sim_values, most_sim_indices = torch.topk(cosine_similarity_with_bias, 10, dim=0)
px.line(sparse_autoencoder.W_dec[most_sim_indices].detach().cpu().T).show()
least_sim_values, least_sim_indices = torch.topk(cosine_similarity_with_bias, 10, dim=0, largest=False)
px.line(sparse_autoencoder.W_dec[least_sim_indices].detach().cpu().T).show()

In [ ]:
normal_examples = ((cosine_similarity_with_bias > -0.01) & (cosine_similarity_with_bias < 0.01)).nonzero(as_tuple=True)[0][:10]
px.line(sparse_autoencoder.W_dec[normal_examples].detach().cpu().mean(dim=0, keepdim=True).T)

In [ ]:
weird_peak_examples = ((cosine_similarity_with_bias > 0.09) & (cosine_similarity_with_bias < 0.11)).nonzero(as_tuple=True)[0][:10]
px.line(sparse_autoencoder.W_dec[weird_peak_examples].detach().cpu().mean(dim=0).unsqueeze(0).T)

In [ ]:
px.line(sparse_autoencoder.W_dec[:8192].detach().abs().cpu().mean(dim=0).unsqueeze(0).T)

In [ ]:
px.line(
    sparse_autoencoder.W_dec[:8192].detach().abs().cpu().mean(dim=0).unsqueeze(0).T -sparse_autoencoder.W_dec[8192:].detach().cpu().abs().mean(dim=0).unsqueeze(0).T)

## End Tangent

In [ ]:

fig = px.scatter(
    y = query_proj_dec_weights_df["L5H5"],
    x = range(len(query_proj_dec_weights_df)),
    hover_name=query_proj_dec_weights_df.index,
    opacity=0.5,
    marginal_x="histogram",
    marginal_y="histogram",
    labels={
        "y": "Feature Norm after Query Projection",
        "x": "Index",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:

fig = px.scatter(
    x = query_proj_dec_weights_df["L10H6"],
    y = log_feature_sparsity,
    hover_name=query_proj_dec_weights_df.index,
    opacity=0.5,
    marginal_x="histogram",
    marginal_y="histogram",
    labels={
        "x": "Feature Norm after Query Projection",
        "y": "Sparsity",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:

fig = px.scatter(
    x = key_proj_dec_weights_df["L10H7"],
    y = log_feature_sparsity,
    hover_name=query_proj_dec_weights_df.index,
    opacity=0.5,
    marginal_x="histogram",
    marginal_y="histogram",
    labels={
        "x": "Feature Norm after Query Projection",
        "y": "Sparisity",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:

fig = px.scatter(
    x = query_proj_dec_weights_df["L10H9"],
    y = key_proj_dec_weights_df["L10H9"],
    hover_name=query_proj_dec_weights_df.index,
    color = [i for i in range(sparse_autoencoder.cfg.d_sae)],# log_feature_sparsity,
    opacity=0.5,
    marginal_x="histogram",
    marginal_y="histogram",
    labels={
        "x": "Feature Norm after Query Projection",
        "y": "Feature Norm after Key Projection",
        "color": "Sparsity",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:
# list all the files in the directy "gpt2_small_features"
import os

feature_list = [int(i.split(".html")[0].split("data_")[1]) for i in os.listdir("./gpt2_small_features")]
feature_list_df = pd.DataFrame(
    {"features": feature_list,
    "sparsity": log_feature_sparsity[feature_list],
    "query_norm": query_proj_dec_weights_df["L10H7"][feature_list],
    "key_norm": key_proj_dec_weights_df["L10H7"][feature_list]}
)
feature_list_df.sort_values("sparsity").head(10)


fig = px.scatter_matrix(
    feature_list_df,
    dimensions=["query_norm", "key_norm", "sparsity"],
    color = feature_list_df["features"],
    hover_name = feature_list_df["features"],
    opacity=0.5,
    # marginal_x="histogram",
    # marginal_y="histogram",
    labels={
        "x": "Feature Norm after Query Projection",
        "y": "Feature Norm after Key Projection",
        "color": "Sparsity",
    },
)
fig.update_layout(width = 1200, height = 600)
fig.show()

In [ ]:
px.histogram(sparse_autoencoder.W_dec.norm(dim=-1).detach().cpu().numpy())

In [ ]:
qk_full = sparse_autoencoder.W_dec[feature_list] @ W_QK_10 @ sparse_autoencoder.W_dec[feature_list].T
fig = px.imshow(qk_full.detach().cpu().numpy(), animation_frame=0,
          color_continuous_scale='RdBu', color_continuous_midpoint=0, title = "Query Key Correlation")

# add feature labels to ticks
fig.update_xaxes(ticktext=feature_list_df["features"], tickvals=feature_list_df.index)
fig.update_yaxes(ticktext=feature_list_df["features"], tickvals=feature_list_df.index)

fig.update_layout(width = 800, height = 1200)
fig.show()

# Summary Stats

In [ ]:
def replacement_hook(mlp_post, hook, encoder):
    mlp_post_reconstr = encoder(mlp_post)[0]
    return mlp_post_reconstr
hook_point = activation_store.cfg.hook_point


with torch.no_grad():
    batch_tokens = activation_store.get_batch_tokens()
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
    original_act = cache[sparse_autoencoder.cfg.hook_point]#[:,:,sparse_autoencoder.cfg.hook_point_head_index]
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        original_act
    )

In [ ]:
px.line(original_act.flatten(0,1).mean(0).detach().cpu().numpy(), title="Original Activation by Position").show()

In [ ]:
tokens = model.to_str_tokens(batch_tokens.flatten())
len(tokens)

In [ ]:
original_act.flatten(0,1)[:,outlier_dimensions].detach().cpu().shape

In [ ]:
outlier_dimensions = (original_act.flatten(0,1).mean(0).detach().cpu().abs() >20).nonzero().squeeze().tolist()
control_dimensions = (original_act.flatten(0,1).mean(0).detach().cpu().abs() <1).nonzero().squeeze().tolist()[:len(outlier_dimensions)]
df1 = pd.DataFrame(
    original_act.flatten(0,1)[:,outlier_dimensions].detach().cpu().numpy(),
    columns = [f"outlier_dim_{i}" for i in outlier_dimensions],
    index = tokens)

df2 = pd.DataFrame(
    original_act.flatten(0,1)[:,control_dimensions].detach().cpu().numpy(),
    columns = [f"control_dim_{i}" for i in control_dimensions],
    index = tokens)

df = pd.concat([df1, df2], axis=1)
px.strip(df[df.index != "<|endoftext|>"], log_y=True, hover_name=df.index[df.index != "<|endoftext|>"]).show()

In [ ]:
print(outlier_dimensions)

In [ ]:
px.histogram(sparse_autoencoder.W_dec[:,outlier_dimensions].detach().cpu(), barmode='overlay', log_y=False,).show()
px.histogram(sparse_autoencoder.W_dec[:,control_dimensions].detach().cpu(), barmode='overlay', log_y=False,).show()

In [ ]:
px.scatter_matrix()

In [ ]:
px.histogram(model.W_out[9,:,outlier_dimensions].detach().cpu(), barmode='overlay', log_y=False,).show()

In [ ]:
px.histogram(model.W_out[9,:,control_dimensions].detach().cpu(), barmode='overlay', log_y=False,).show()

In [ ]:
cache[utils.get_act_name("mlp_out", 9)].flatten()

# Random Token Experiment

In [ ]:
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=5, n_repeat_tokens=2)
prompt = model.to_string(random_tokens)
token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt], model, sparse_autoencoder, head_idx_override=7)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].tail(10).style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm",
)

In [ ]:
HEAD_IDX = 7
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
POS_INTEREST = feature_acts.shape[0] - 1 # index from 0.

# reload(joseph.visualisation)
# from joseph.visualisation import *

print(token_df["unique_token"][POS_INTEREST]) # bos gone.
plot_line_with_top_10_labels(feature_acts[POS_INTEREST], "", 10)

vals, inds = torch.topk(feature_acts[POS_INTEREST],5)

print(inds)
# plot_attn_score_by_feature(model, sparse_autoencoder, inds, original_cache, token_df, pos_interest=POS_INTEREST, vals = vals)
# plot_unembed_score_by_feature(model, sparse_autoencoder, inds, token_df, vals=vals)

In [ ]:
# set pandas columns allowed to be as wide as they want.
pd.set_option('display.max_colwidth', None)
filter_columns = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
# features = torch.randint(0, sparse_autoencoder.cfg.d_sae, (50,))
# features = torch.tensor([24519,22818,31246])
for feature in inds:
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(10))

In [ ]:
data = get_webtext()

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []
# data = get_webtext()

NUM_PROMPTS = 30
# MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
feature_acts_list = []
with torch.no_grad():
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = data[i]
        # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
        

        token_df, _, _, feature_acts = eval_prompt(prompt, model, sparse_autoencoder, head_idx_override=7)
        feature_acts_list.append(feature_acts)
        dataframe_list.append(token_df)
        
all_token_df = pd.concat(dataframe_list)
all_token_df.reset_index(drop=True)
all_token_features = torch.cat(feature_acts_list)

print(all_token_df.shape)
print(all_token_df.columns)
all_token_df.head()

In [ ]:
pos_mask = (all_token_df.pos.values < 128)
all_token_df = all_token_df[all_token_df.pos < 128]
all_token_features = all_token_features[torch.tensor(pos_mask)]

num_active_features_mask = (all_token_df.num_active_features.values < 100)
all_token_df = all_token_df[all_token_df.num_active_features < 100]
all_token_features = all_token_features[torch.tensor(num_active_features_mask)]

In [ ]:
sparsity = (all_token_features > 0).float().mean(dim=0).detach().cpu()
log_sparsity = torch.log10(sparsity)
print((sparsity > 0).float().mean())
print((sparsity > 0).float().sum())

In [ ]:
px.histogram(log_sparsity, nbins=100).show()
px.strip(log_sparsity, hover_data=[[i for i in range(sparse_autoencoder.cfg.d_sae)]]).show()

In [ ]:
all_token_df.columns

In [ ]:
px.histogram(all_token_df.num_active_features)

In [ ]:
filter_columns = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
features = torch.randint(0, sparse_autoencoder.cfg.d_sae, (5,))
# features = torch.tensor([24519,22818,31246])
for feature in features:
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.0001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(10))
    # else:``
        # print(f"feature {feature.item()} is dead")

# Specific Example

In [ ]:
prompt = "When John and Mary went to the shops, John gave the shopping to"
answer = " Mary"
# prompt = "All's fair in love and"
# answer = " war"
# prompt = " The cat is cute. The dog is"
# prompt = " Alice, with her keen intelligence and artistic talent, discussed philosophy with Bob, who shared her intellect and also possessed remarkable culinary skills, while"
# answer = " cute"
model.reset_hooks()
utils.test_prompt(prompt, answer, model)

HEAD_HOOK_RESULT_NAME = "blocks.10.attn.hook_z"
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
HEAD_IDX = 7
def hook_to_ablate_head(head_output: Float[Tensor, "batch seq_len head_idx d_head"], hook: HookPoint, head = (LAYER_IDX, HEAD_IDX)):
    print(hook.layer(), hook.name)
    assert head[0] == hook.layer(), f"{head[0]} != {hook.layer()}"
    assert ("result" in hook.name) or ("q" in hook.name) or ("z" in hook.name)
    head_output[:, :, head[1], :] = 0
    return head_output

with model.hooks(fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)]):
    utils.test_prompt(prompt, answer, model)

In [ ]:
import joseph
reload(joseph.analysis)
from joseph.analysis import *


token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt + answer], model, sparse_autoencoder, head_idx_override=7)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence",
               "top_k_features"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm")


In [ ]:
(feature_acts > 0).float().sum(1)

In [ ]:
def plot_attn(patterns, token_df, title="", facet_col_labels = ["Original", "Reconstructed"]):
    '''
    # patterns_original = cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    # patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_original = cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    both_patterns = torch.stack([patterns_original, patterns_reconstructed])
    plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
    
    '''
    fig = px.imshow(patterns, text_auto=".2f", title=title,
                    facet_col=0,
                    color_continuous_midpoint=0,
                    color_continuous_scale="RdBu",
                    )
    
    tickvals = np.arange(patterns.shape[2])
    ticktext = token_df["unique_token"].tolist()
    
    # add tokens as x-ticks and y-ticks, for each facet
    # Update x-ticks and y-ticks for each facet
    for i in range(len(facet_col_labels)):
        fig.update_xaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
        fig.update_yaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
    
    
    # add facet col labels:
    for i, label in enumerate(facet_col_labels):
        fig.layout.annotations[i].text = label
        fig.layout.annotations[i].font.size = 20
        
    fig.update_layout(
        width=1200,
        height=800,
    )
    fig.show()


LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
HEAD_IDX = 7
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
POS_INTEREST = 14# index from 0.
print(token_df.shape)
print(feature_acts.shape)
print(token_df["unique_token"][POS_INTEREST]) 
feature_acts_of_interest = feature_acts[POS_INTEREST]
plot_line_with_top_10_labels(feature_acts_of_interest, "", 30)
vals, inds = torch.topk(feature_acts_of_interest,64)
print(vals.nonzero().shape)
print(inds)
# plot_attn_score_by_feature(model, sparse_autoencoder, inds, original_cache, token_df, pos_interest=POS_INTEREST, vals = vals)
# plot_unembed_score_by_feature(model, sparse_autoencoder, inds, token_df, vals=vals)

In [ ]:
px.strip(
    (sparse_autoencoder.W_dec @ model.W_U[:,model.to_single_token(" Mary")]).detach().cpu().numpy(), 
    color = [i in inds for i in range(sparse_autoencoder.cfg.d_sae)],
    hover_data=[[i for i in range(sparse_autoencoder.cfg.d_sae)]],
    ).show()


In [ ]:
# set pandas not to truncate witdth 
pd.set_option('display.max_colwidth', None)
# set pandas not to use elipses 
# pd.set_option('display.max_cols', None)
k = 64
virtual_weights = (sparse_autoencoder.W_dec[inds]  @ model.W_U)
top_k_tokens = virtual_weights.topk(k, dim=-1).indices
tokens = pd.DataFrame(top_k_tokens.detach().cpu().T, columns = [f"feature_{i}" for i in inds], index = [f"top_{i}" for i in range(k)])
df = tokens.applymap(lambda x: model.to_string(x))
df.to_csv("test.csv")

In [ ]:
px.bar(
    x = [str(i.item()) for i in inds],
    y = (sparse_autoencoder.W_dec[inds] @ model.W_U[:,model.to_single_token(" Mary")]).detach().cpu().numpy()).show()

px.bar(
    x = [str(i.item()) for i in inds],
    y = (sparse_autoencoder.W_dec[inds] @ model.W_U[:,model.to_single_token(" John")]).detach().cpu().numpy()).show()

px.bar(
    x = [str(i.item()) for i in inds],
    y = (sparse_autoencoder.W_dec[inds] @ (model.W_U[:,model.to_single_token(" Mary")] - model.W_U[:,model.to_single_token(" John")] )).detach().cpu().numpy())


In [ ]:
df_list = [] 
for layer in range(8,12):
    
    logit_lens_at_layer_10 = original_cache.apply_ln_to_stack(original_cache[utils.get_act_name("resid_pre", layer)]) 
    logit_lens_at_layer_10 = logit_lens_at_layer_10[0, POS_INTEREST] @ model.W_U
    vals, inds = torch.topk(logit_lens_at_layer_10.detach().cpu(), 50)
    df_list.append(pd.DataFrame({f"token_{layer}": [model.to_string(i) for i in inds], f"logit_len_{layer}": vals}).head(30))

pd.concat(df_list, axis=1).style.background_gradient(
    subset=[f"logit_len_{layer}" for layer in range(8,12)],
    cmap="coolwarm",
)


In [ ]:
def plot_attn_score_by_feature(
    model: HookedTransformer, 
    sparse_autoencoder: SparseAutoencoder, 
    feature_ids, 
    cache, 
    token_df, 
    pos_interest, 
    vals = None,
    head_index = None):
    '''
    '''
    
    layer_index = sparse_autoencoder.cfg.hook_point_layer
    head_index = sparse_autoencoder.cfg.hook_point_head_index if head_index is None else head_index

    k = cache[f"blocks.{layer_index}.attn.hook_k"][0,:(1+pos_interest),head_index]
    # score_contributions = sparse_autoencoder.W_enc[:,inds].T @ k.T
    score_contributions = (sparse_autoencoder.W_dec[feature_ids] @ model.W_Q[10,7]) @ k.T

    if vals is not None:
        score_contributions = score_contributions.cpu() * vals.unsqueeze(1).cpu()
    fig = px.imshow(score_contributions.detach().cpu(), 
                    color_continuous_scale="RdBu",
                    color_continuous_midpoint=0,
                    labels = dict(y="Feature", x="Token"),
                    text_auto=".2f", title="")
    # add xticks and y ticks
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=1+np.arange(score_contributions.shape[1]),
            ticktext=token_df["str_tokens"].tolist(),
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=np.arange(score_contributions.shape[0]),
            ticktext=list(feature_ids.detach().cpu().numpy()),
        ),
    )
    return fig

print(POS_INTEREST)
fig = plot_attn_score_by_feature(model, sparse_autoencoder, inds[:10], original_cache, token_df, pos_interest=POS_INTEREST+1, head_index = 7, vals = vals[:10])
fig.update_layout(width = 1000, height = 1200)
fig

In [ ]:
sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(original_cache[sparse_autoencoder.cfg.hook_point])
query_features_firing = (feature_acts[0, POS_INTEREST] > 0)
print(query_features_firing.sum())

In [ ]:
key_feature_projections = (feature_acts[0, :-1] @ sparse_autoencoder.W_dec) @ model.W_K[10,7]
tmp = key_feature_projections @ cache_reconstructed_query["blocks.10.attn.hook_q"][0, POS_INTEREST, 7].T
px.line(
    x = token_df.unique_token.tolist()[1:-1],
    y = tmp[1:].detach().cpu().numpy()).show()

In [ ]:
interesting_features = []

In [ ]:
key_pos = 2
query_pos = POS_INTEREST

key_features_firing = (feature_acts[0, key_pos] > 0)
print(key_features_firing.sum())

query_features_firing = (feature_acts[0, query_pos] > 0)
print(query_features_firing.sum())

key_features = (feature_acts[0, key_pos, key_features_firing] * sparse_autoencoder.W_dec[key_features_firing].T[:, None, :]).transpose(2,0)
print(key_features.shape)
query_features = (feature_acts[0, query_pos, query_features_firing] * sparse_autoencoder.W_dec[query_features_firing].T)
print(query_features.shape)
scores = key_features @ query_features
df = pd.DataFrame(scores[:,0].detach().cpu().numpy(), 
                  index = key_features_firing.nonzero().squeeze().tolist(), 
                  columns = query_features_firing.nonzero().squeeze().tolist())
fig = px.imshow(df.values, 
                title = " Key / Query Feature Scores (ignore LN) for Mary / to",
                color_continuous_midpoint=0, 
                color_continuous_scale="RdBu")
# update xticks and yticks
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(df.shape[1]),
        ticktext=list(df.columns),
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=np.arange(df.shape[0]),
        ticktext=list(df.index),
    ),
)
fig.show()

In [ ]:
interesting_features = interesting_features + key_features_firing.nonzero().squeeze().tolist()
interesting_features = interesting_features + query_features_firing.nonzero().squeeze().tolist()
interesting_features = list(set(interesting_features))
len(interesting_features)

In [ ]:
key_pos = 4
query_pos = POS_INTEREST

key_features_firing = (feature_acts[0, key_pos] > 0)
print(key_features_firing.sum())

query_features_firing = (feature_acts[0, query_pos] > 0)
print(query_features_firing.sum())

key_features = (feature_acts[0, key_pos, key_features_firing] * sparse_autoencoder.W_dec[key_features_firing].T[:, None, :]).transpose(2,0)
print(key_features.shape)
query_features = (feature_acts[0, query_pos, query_features_firing] * sparse_autoencoder.W_dec[query_features_firing].T)
print(query_features.shape)
scores = key_features @ query_features
df = pd.DataFrame(scores[:,0].detach().cpu().numpy(), 
                  index = key_features_firing.nonzero().squeeze().tolist(), 
                  columns = query_features_firing.nonzero().squeeze().tolist())
fig = px.imshow(df.values, 
                title = " Key / Query Feature Scores (ignore LN) for Mary / to",
                color_continuous_midpoint=0, 
                color_continuous_scale="RdBu")
# update xticks and yticks
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(df.shape[1]),
        ticktext=list(df.columns),
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=np.arange(df.shape[0]),
        ticktext=list(df.index),
    ),
)
fig.show()

In [ ]:
interesting_features = interesting_features + key_features_firing.nonzero().squeeze().tolist()
interesting_features = interesting_features + query_features_firing.nonzero().squeeze().tolist()
interesting_features = list(set(interesting_features))
len(interesting_features)

In [ ]:
all_tokens.shape

## Interventions on Features

We want to experiment with deleting a feature. To do this with the resid pre SAE, we need to generate the query from the resid_pre and then minus the feature and then patch the hook q with the new hook_q.

In [ ]:

prompt = "When John and Mary went to the shops, John gave the shopping to"
answer = " Mary"
tokens = model.to_tokens(prompt)

token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt(prompt, model, sparse_autoencoder, head_idx_override=7)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence",
               "top_k_features"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm")


In [ ]:
vals, inds = torch.topk(feature_acts[-1],30)

In [ ]:
layer_idx = sparse_autoencoder.cfg.hook_point_layer
head_idx = 7
head_hook_query_name = utils.get_act_name("q", layer_idx)


attn_df = make_token_df(model, tokens)
patterns = original_cache[f"blocks.{layer_idx}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
attn_df["original_attn"] = patterns[-1,]
patterns = cache_reconstructed_query[f"blocks.{layer_idx}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
attn_df["reconstructed_attn"] = patterns[-1,]

for feature in inds:
    features_to_remove = [feature]
    original_act = original_cache[sparse_autoencoder.cfg.hook_point]
    sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_act)

    # need to generate query
    def replacement_hook(resid_pre, hook, new_resid_pre=sae_out):
        return new_resid_pre

    with model.hooks(fwd_hooks=[(utils.get_act_name("resid_pre", sparse_autoencoder.cfg.hook_point_layer), replacement_hook)]):
        _, resid_pre_cache = model.run_with_cache(tokens, return_type="loss", loss_per_token=True)
        sae_out = resid_pre_cache[head_hook_query_name][:,:,head_idx]


    def remove_feature_hook(hook_in, hook, position=-1, features_to_remove = features_to_remove):
        for feature_to_remove in features_to_remove:
            # print(feature_acts[0,position,feature_to_remove].item())
            feature_dir = feature_acts[0,position,feature_to_remove]*sparse_autoencoder.W_dec[feature_to_remove]
            hook_in -= feature_dir
        return hook_in

    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]):
        _, cache_removed_feature = model.run_with_cache(tokens, return_type="loss", loss_per_token=True)

    patterns = cache_removed_feature[f"blocks.{layer_idx}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df[f"ablated_feature_{feature}"] = patterns[-1,]

fig = px.line(attn_df, 
            x="unique_token",
            y=["original_attn","reconstructed_attn"] + [f"ablated_feature_{feature}" for feature in inds],
            hover_name="str_tokens", 
            hover_data=["pos", "batch", "label"], 
            title="Original vs Reconstructed attention")

# increase figure height
fig.update_layout(height=800)
fig.show()





In [ ]:
import plotly.graph_objects as go
fig = px.bar(
    x= attn_df.T[[2,4]][8:].index,
    y=attn_df.T.iloc[8:,4] / attn_df.T.iloc[8:,2],
    hover_name= attn_df.T[[2,4]][8:].index,
    labels= {"x": "Ablated Feature", "y": "Mary Attn / John Attn"})
print(attn_df.T.iloc[7,4] / attn_df.T.iloc[7,2])
fig.show()

In [ ]:

fig = px.line(attn_df, 
            x="unique_token",
            y=["original_attn","reconstructed_attn"] + [f"ablated_feature_{feature}" for feature in inds],
            hover_name="str_tokens", 
            hover_data=["pos", "batch", "label", "variable"], 
            title="Original vs Reconstructed attention")

# increase figure height
fig.update_layout(height=800)
fig.show()


In [ ]:
px.line(attn_df.iloc[4,6:])

In [ ]:
px.bar(x = attn_df.iloc[4, 8:].index, y = attn_df.iloc[4, 8:].values).show()

# Feature DLA

In [ ]:
decomp, labels = original_cache.get_full_resid_decomposition(layer =  10, expand_neurons=False, return_labels=True)
print(decomp[:,0,-1].shape)

test = (decomp[:,0,-1] @ sparse_autoencoder.W_enc[:,inds])
# test = (decomp[:,0,-1] @ sparse_autoencoder.W_dec[inds].T) / sparse_autoencoder.W_enc[:,inds].norm(dim=0)
tmp = pd.DataFrame(test.detach().cpu().numpy().T, columns = labels, index = [f"feature_{i}" for i in inds])
px.line(
    tmp.T[tmp.T.index.str.contains("mlp")]
).show()

px.line(
    tmp.T[tmp.T.index.str.contains("L")]
).show()

# test = (decomp[:,0,-1] @ sparse_autoencoder.W_enc[:,inds])
test = (decomp[:,0,-1] @ sparse_autoencoder.W_dec[inds].T) / sparse_autoencoder.W_enc[:,inds].norm(dim=0)
tmp = pd.DataFrame(test.detach().cpu().numpy().T, columns = labels, index = [f"feature_{i}" for i in inds])
px.line(
    tmp.T[tmp.T.index.str.contains("mlp")]
).show()
px.line(
    tmp.T[tmp.T.index.str.contains("L")]
).show()

# Quick Check for Outlier Dims

In [ ]:
all_token_features.shape

In [ ]:
firing_features = feature_acts[-1] > 0
firing_features_labels = [f"feature_{i}" for i in firing_features.nonzero().squeeze().tolist()]
tmp = pd.DataFrame( sparse_autoencoder.W_dec[firing_features].detach().cpu(), index = firing_features_labels)
px.line(
   tmp.T,
).show()

In [ ]:
all_token_features

In [ ]:
sae_out = feature_acts[-1] @ sparse_autoencoder.W_dec + sparse_autoencoder.b_dec

for j in range(10,11):
    px.line(
        torch.stack(
            [original_cache[utils.get_act_name("resid_pre", i)][0,j].detach().cpu().squeeze() for i in range(12)]).squeeze(0).T).show()
px.line(sae_out.detach().cpu()).show() 

In [ ]:
px.line((all_token_features > 0).float().mean(0).detach().cpu()).show()

# Cluster Features

In [ ]:
from scipy.cluster import hierarchy
sim = sparse_autoencoder.W_dec[inds] @ sparse_autoencoder.W_dec[inds].T
sim = sim.detach().cpu().numpy()
labels = [f"feature_{i}" for i in inds]
linkage = hierarchy.linkage(sim)
dendrogram = hierarchy.dendrogram(linkage, no_plot=True, color_threshold=-np.inf)
reordered_ind = list(reversed(dendrogram["leaves"]))
reordered_labels = [labels[i] for i in reordered_ind]
reordered_arr = sim[reordered_ind][:, reordered_ind]
px.imshow(reordered_arr, x=reordered_labels, y=reordered_labels, color_continuous_midpoint=0, color_continuous_scale="RdBu").show()

In [ ]:
from scipy.cluster import hierarchy
labels = [f"feature_{i}" for i in inds]
linkage = hierarchy.linkage(sim)
dendrogram = hierarchy.dendrogram(linkage, no_plot=True, color_threshold=-np.inf)
reordered_ind = list(reversed(dendrogram["leaves"]))
reordered_labels = [labels[i] for i in reordered_ind]
reordered_arr = sim[reordered_ind][:, reordered_ind]
px.imshow(reordered_arr, x=reordered_labels, y=reordered_labels, color_continuous_midpoint=0, color_continuous_scale="RdBu").show()

In [ ]:
sparse_autoencoder.b_dec.shape

In [ ]:
sparse_autoencoder.b_enc.shape

## Scratch

In [ ]:
sae_out.shape


In [ ]:
from geom_median.torch import compute_geometric_median
out = compute_geometric_median(sae_out[0].detach().cpu())
out

In [ ]:
sae_out[0].detach().cpu().numpy() - out.median

# Feature Analysis

In [ ]:
data = get_webtext()

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []
# data = get_webtext()

NUM_PROMPTS = 200
# MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
feature_acts_list = []
with torch.no_grad():
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = model.to_string(model.to_tokens(data[i])[0,:128])
        token_df, _, _, feature_acts = eval_prompt(prompt, model, sparse_autoencoder, head_idx_override=7)
        feature_acts_list.append(feature_acts)
        dataframe_list.append(token_df)
        
all_token_df = pd.concat(dataframe_list)
all_token_df.reset_index(drop=True)
all_token_features = torch.cat(feature_acts_list)

print(all_token_df.shape)
print(all_token_df.columns)
all_token_df.head()

In [ ]:
pos_mask = (all_token_df.pos.values < 128)
all_token_df = all_token_df[all_token_df.pos < 128]
all_token_features = all_token_features[torch.tensor(pos_mask)]

sparsity = (all_token_features > 0).float().mean(dim=0).detach().cpu()
log_sparsity = torch.log10(sparsity)
print((sparsity > 0).float().mean())
print((sparsity > 0).float().sum())

num_active_features_mask = (all_token_df.num_active_features.values < 100)
all_token_df = all_token_df[all_token_df.num_active_features < 100]
all_token_features = all_token_features[torch.tensor(num_active_features_mask)]

sparsity = (all_token_features > 0).float().mean(dim=0).detach().cpu()
log_sparsity = torch.log10(sparsity)
print((sparsity > 0).float().mean())
print((sparsity > 0).float().sum())

In [ ]:
all_token_df.shape

In [ ]:
px.histogram(log_sparsity, nbins=100).show()
px.strip(log_sparsity, hover_data=[[i for i in range(sparse_autoencoder.cfg.d_sae)]]).show()
px.histogram(all_token_df.num_active_features)

In [ ]:
all_token_features

In [ ]:
filter_columns = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
features = torch.randint(0, sparse_autoencoder.cfg.d_sae, (5,))
# features = torch.tensor([24519,22818,31246])
for feature in features:
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.0001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(10))
    # else:``
        # print(f"feature {feature.item()} is dead")

# Interventions on Features

# Anthropic Feature Dashboard

In [ ]:
all_tokens_list = []
pbar = tqdm(range(128*6))
for i in pbar:
    all_tokens_list.append(activation_store.get_batch_tokens())
all_tokens = torch.cat(all_tokens_list, dim=0)
print(all_tokens.shape)
all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]

In [ ]:
all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
tokens = all_tokens[:4096*6]
del all_tokens
torch.mps.empty_cache()

In [ ]:
from sae_analysis.visualizer import data_fns, model_fns, html_fns
import importlib

importlib.reload(data_fns)
importlib.reload(html_fns)
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

# Currently, don't think much more time can be squeezed out of it. Maybe the best saving would be to
# make the entire sequence indexing parallelized, but that's possibly not worth it right now.

max_batch_size = 512
total_batch_size = 4096*6
feature_idx = [i for i in range(sparse_autoencoder.cfg.d_sae)]
feature_idx = torch.tensor(feature_idx).reshape(512, -1)
feature_idx = [feature_idx[i].tolist() for i in range(512)]
# max_batch_size = 512
# total_batch_size = 16384
# feature_idx = list(range(1000))


# shuffle

for interesting_features in tqdm(feature_idx):
    print(interesting_features)
    feature_data = get_feature_data(
        encoder=sparse_autoencoder,
        # encoder_B=sparse_autoencoder,
        model=model,
        hook_point=sparse_autoencoder.cfg.hook_point,
        hook_point_layer=sparse_autoencoder.cfg.hook_point_layer,
        hook_point_head_index=None,
        tokens=tokens,
        feature_idx=interesting_features,
        max_batch_size=max_batch_size,
        left_hand_k = 3,
        buffer = (5, 5),
        n_groups = 10,
        first_group_size = 20,
        other_groups_size = 5,
        verbose = True,
    )


    for test_idx in list(interesting_features):
        html_str = feature_data[test_idx].get_all_html()
        with open(f"gpt2_small_features_layer_5/data_{test_idx:04}.html", "w") as f:
            f.write(html_str)

In [ ]:
from sae_analysis.visualizer import data_fns, model_fns, html_fns
import importlib

importlib.reload(data_fns)
importlib.reload(html_fns)
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

# Currently, don't think much more time can be squeezed out of it. Maybe the best saving would be to
# make the entire sequence indexing parallelized, but that's possibly not worth it right now.

max_batch_size = 512
total_batch_size = 4096*6
feature_idx = [i for i in range(sparse_autoencoder.cfg.d_sae)]
feature_idx = torch.tensor(feature_idx).reshape(512, -1)
feature_idx = [feature_idx[i].tolist() for i in range(512)]
# max_batch_size = 512
# total_batch_size = 16384
# feature_idx = list(range(1000))


# shuffle
interesting_features = temp_df

feature_data = get_feature_data(
    encoder=sparse_autoencoder,
    # encoder_B=sparse_autoencoder,
    model=model,
    hook_point=sparse_autoencoder.cfg.hook_point,
    hook_point_layer=sparse_autoencoder.cfg.hook_point_layer,
    hook_point_head_index=None,
    tokens=tokens,
    feature_idx=interesting_features,
    max_batch_size=max_batch_size,
    left_hand_k = 3,
    buffer = (5, 5),
    n_groups = 10,
    first_group_size = 20,
    other_groups_size = 5,
    verbose = True,
)


for test_idx in list(interesting_features):
    html_str = feature_data[test_idx].get_all_html()
    with open(f"gpt2_small_features_layer_5/data_{test_idx:04}.html", "w") as f:
        f.write(html_str)

In [ ]:
temp_df[temp_df.cluster == 15].index[:10]